<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/als_pro_act_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane

In [2]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("STEP 1: INITIALIZATION AND DATA LOADING")
print("=" * 60)

# -------------------------------
# 1. Load all relevant CSV tables
# -------------------------------
print("\n📂 Loading PROACT datasets...")
alsfrs_df = pd.read_csv('PROACT_ALSFRS.csv')
fvc_df = pd.read_csv('PROACT_FVC.csv')
vitals_df = pd.read_csv('PROACT_VITALSIGNS.csv')
labs_df = pd.read_csv('PROACT_LABS.csv')
onset_df = pd.read_csv('PROACT_ALSHISTORY.csv')
riluzole_df = pd.read_csv('PROACT_RILUZOLE.csv')
demographics_df = pd.read_csv('PROACT_DEMOGRAPHICS.csv')

print(f"✅ ALSFRS records: {len(alsfrs_df):,}")
print(f"✅ FVC records: {len(fvc_df):,}")
print(f"✅ Vitals records: {len(vitals_df):,}")
print(f"✅ Labs records: {len(labs_df):,}")
print(f"✅ Demographics: {len(demographics_df):,}")

# -------------------------------
# 2. Compute ALSFRS (convert ALSFRS-R to original if needed)
# -------------------------------
print("\n🧮 Computing ALSFRS scores...")

def convert_alsfrs_row(row):
    """Convert ALSFRS-R to original ALSFRS if needed"""
    if pd.notna(row.get('ALSFRS_Total')):
        return row['ALSFRS_Total']
    total = 0
    for q in range(1, 10):
        val = row.get(f'Q{q}', np.nan)
        if pd.notna(val):
            total += val
    # Handle Q10 (respiratory)
    if pd.notna(row.get('Q10_Respiratory')):
        total += row['Q10_Respiratory']
    elif pd.notna(row.get('R_1_Dyspnea')):
        total += row.get('R_1_Dyspnea')
    return total

alsfrs_df['ALSFRS_Total_orig'] = alsfrs_df.apply(convert_alsfrs_row, axis=1)

# -------------------------------
# 3. Identify valid patients
# -------------------------------
print("\n🔍 Identifying valid patients...")
months_start, months_end = 3, 12
min_records_start, min_records_end = 2, 2
days_start, days_end = months_start * 30, months_end * 30

alsfrs_counts = alsfrs_df.groupby('subject_id')['ALSFRS_Delta'].agg(
    records_before_start=lambda x: (x <= days_start).sum(),
    records_after_end=lambda x: (x >= days_end).sum()
)

valid_patients_df = alsfrs_counts[
    (alsfrs_counts['records_before_start'] >= min_records_start) &
    (alsfrs_counts['records_after_end'] >= min_records_end)
]
valid_patients = sorted(valid_patients_df.index.tolist())

print(f"✅ Valid patients identified: {len(valid_patients):,}")

# -------------------------------
# 4. Compute ALSFRS slope (3–12 months) - TARGET
# -------------------------------
print("\n📈 Computing target ALSFRS slope (3-12 months)...")
slope_targets = {}

for pid in valid_patients:
    patient_data = alsfrs_df[alsfrs_df['subject_id'] == pid].copy()
    patient_data.sort_values('ALSFRS_Delta', inplace=True)
    t1 = patient_data[patient_data['ALSFRS_Delta'] > 90]
    t2 = patient_data[patient_data['ALSFRS_Delta'] >= 365]

    if len(t1) > 0 and len(t2) > 0:
        t1_record = t1.iloc[0]
        t2_record = t2.iloc[0]
        delta_days = t2_record['ALSFRS_Delta'] - t1_record['ALSFRS_Delta']
        if delta_days > 0:
            slope = (t2_record['ALSFRS_Total_orig'] - t1_record['ALSFRS_Total_orig']) / (delta_days / 30.0)
            slope_targets[pid] = slope

target_df = pd.Series(slope_targets, name='ALSFRS_slope_3to12m')
print(f"✅ ALSFRS slope computed for {len(target_df):,} patients")
print("\n📊 Target Statistics:")
print(target_df.describe())

# -------------------------------
# 5. Helper functions for feature engineering
# -------------------------------
print("\n🛠️  Setting up feature engineering functions...")

def summarize_timeseries(df, time_col, value_col):
    """Enhanced time-series summarization with additional statistics"""
    grp = df.groupby('subject_id')
    summary = pd.DataFrame({
        'min': grp[value_col].min(),
        'max': grp[value_col].max(),
        'mean': grp[value_col].mean(),  # Added mean
        'median': grp[value_col].median(),
        'std': grp[value_col].std(),
        'q25': grp[value_col].quantile(0.25),  # Added 25th percentile
        'q75': grp[value_col].quantile(0.75),  # Added 75th percentile
        'first': grp.apply(lambda g: g.sort_values(time_col)[value_col].iloc[0], include_groups=False),
        'last': grp.apply(lambda g: g.sort_values(time_col)[value_col].iloc[-1], include_groups=False)
    })

    # Compute slope (rate of change)
    time_first = grp[time_col].min()
    time_last = grp[time_col].max()
    time_diff_months = (time_last - time_first) / 30.0
    summary['slope'] = (summary['last'] - summary['first']) / time_diff_months
    summary.loc[time_diff_months == 0, 'slope'] = np.nan

    # Add range
    summary['range'] = summary['max'] - summary['min']

    return summary

def summarize_all_numeric(df, time_col):
    """Summarize all numeric columns in a time-series DataFrame"""
    numeric_cols = df.select_dtypes(include=['number']).columns.drop([time_col, 'subject_id'], errors='ignore')
    summaries = {}
    for col in numeric_cols:
        summaries[col] = summarize_timeseries(df, time_col, col)
        summaries[col].columns = [f'{col}_{c}' for c in summaries[col].columns]
    return summaries

print("✅ Feature engineering functions ready")

# -------------------------------
# 6. Extract first 90 days data
# -------------------------------
print("\n📅 Extracting first 90 days data...")

alsfrs_3m = alsfrs_df[alsfrs_df['subject_id'].isin(valid_patients) & (alsfrs_df['ALSFRS_Delta'] <= 90)]
fvc_df['FVC'] = fvc_df[['Subject_Liters_Trial_1','Subject_Liters_Trial_2','Subject_Liters_Trial_3']].max(axis=1)
fvc_3m = fvc_df[fvc_df['subject_id'].isin(valid_patients) & (fvc_df['Forced_Vital_Capacity_Delta'] <= 90)]
vitals_3m = vitals_df[vitals_df['subject_id'].isin(valid_patients) & (vitals_df['Vital_Signs_Delta'] <= 90)]
labs_3m = labs_df[labs_df['subject_id'].isin(valid_patients) & (labs_df['Laboratory_Delta'] <= 90)]

print(f"✅ ALSFRS 3-month records: {len(alsfrs_3m):,}")
print(f"✅ FVC 3-month records: {len(fvc_3m):,}")
print(f"✅ Vitals 3-month records: {len(vitals_3m):,}")
print(f"✅ Labs 3-month records: {len(labs_3m):,}")

# -------------------------------
# 7. Create summarized features
# -------------------------------
print("\n🔨 Creating summarized features from time-series data...")

alsfrs_features = summarize_all_numeric(alsfrs_3m, 'ALSFRS_Delta')
fvc_features = summarize_all_numeric(fvc_3m, 'Forced_Vital_Capacity_Delta')
vitals_features = summarize_all_numeric(vitals_3m, 'Vital_Signs_Delta')
labs_features = summarize_all_numeric(labs_3m, 'Laboratory_Delta')

print(f"✅ ALSFRS features: {len(alsfrs_features)} variables")
print(f"✅ FVC features: {len(fvc_features)} variables")
print(f"✅ Vitals features: {len(vitals_features)} variables")
print(f"✅ Labs features: {len(labs_features)} variables")

# -------------------------------
# 8. Merge all features
# -------------------------------
print("\n🔗 Merging all features...")

features_df = pd.DataFrame(index=valid_patients)

# Prepare static features
onset_static = onset_df.drop_duplicates(subset='subject_id', keep='first').set_index('subject_id')[['Site_of_Onset', 'Onset_Delta', 'Diagnosis_Delta']]
riluzole_static = riluzole_df.drop_duplicates(subset='subject_id', keep='first').set_index('subject_id')[['Subject_used_Riluzole', 'Riluzole_use_Delta']]
demographics_static = demographics_df.drop_duplicates(subset='subject_id', keep='first').set_index('subject_id')[['Age', 'Sex']]

# Join static features
features_df = features_df.join(onset_static, how='left')
features_df = features_df.join(riluzole_static, how='left', rsuffix='_rilu')
features_df = features_df.join(demographics_static, how='left', rsuffix='_demo')

# Add dynamic (summarized) features
for group in [alsfrs_features, fvc_features, vitals_features, labs_features]:
    for feat_df in group.values():
        features_df = features_df.join(feat_df, how='left')

# Add slope target
features_df = features_df.join(target_df, how='left')

print(f"✅ Features merged. Shape: {features_df.shape}")

# -------------------------------
# 9. Initial cleanup
# -------------------------------
print("\n🧹 Initial cleanup...")

# Remove columns with all NaN values
features_df = features_df.dropna(axis=1, how='all')

# Remove columns with only one unique value
features_df = features_df.loc[:, features_df.nunique() > 1]

print(f"✅ After cleanup. Shape: {features_df.shape}")
print(f"\n📋 Missing values per column: {features_df.isnull().sum().sum():,} total")
print(f"📋 Columns with >30% missing: {(features_df.isnull().sum() / len(features_df) > 0.3).sum()}")

print("\n" + "=" * 60)
print("✅ STEP 1 COMPLETED SUCCESSFULLY")
print("=" * 60)
print(f"\n📊 Final dataset info:")
print(f"  - Total patients: {len(features_df):,}")
print(f"  - Total features: {features_df.shape[1] - 1}")  # -1 for target
print(f"  - Target variable: ALSFRS_slope_3to12m")
print(f"  - Patients with target: {features_df['ALSFRS_slope_3to12m'].notna().sum():,}")
print("\n🔍 Preview:")
print(features_df.head(3))

STEP 1: INITIALIZATION AND DATA LOADING

📂 Loading PROACT datasets...
✅ ALSFRS records: 73,845
✅ FVC records: 49,110
✅ Vitals records: 84,721
✅ Labs records: 2,937,162
✅ Demographics: 12,504

🧮 Computing ALSFRS scores...

🔍 Identifying valid patients...
✅ Valid patients identified: 2,442

📈 Computing target ALSFRS slope (3-12 months)...
✅ ALSFRS slope computed for 2,439 patients

📊 Target Statistics:
count    2439.000000
mean       -0.388076
std         0.496497
min        -3.100000
25%        -0.638298
50%        -0.218978
75%         0.000000
max         1.052632
Name: ALSFRS_slope_3to12m, dtype: float64

🛠️  Setting up feature engineering functions...
✅ Feature engineering functions ready

📅 Extracting first 90 days data...
✅ ALSFRS 3-month records: 8,210
✅ FVC 3-month records: 5,880
✅ Vitals 3-month records: 9,625
✅ Labs 3-month records: 403,408

🔨 Creating summarized features from time-series data...
✅ ALSFRS features: 17 variables
✅ FVC features: 8 variables
✅ Vitals features: 27

In [3]:
features_df.head(3)

,Site_of_Onset,Onset_Delta,Diagnosis_Delta,Subject_used_Riluzole,Riluzole_use_Delta,Age,Sex,Q1_Speech_min,Q1_Speech_max,Q1_Speech_mean,...,Standing_BP_Diastolic_last,Standing_BP_Systolic_min,Standing_BP_Systolic_max,Standing_BP_Systolic_mean,Standing_BP_Systolic_median,Standing_BP_Systolic_q25,Standing_BP_Systolic_q75,Standing_BP_Systolic_first,Standing_BP_Systolic_last,ALSFRS_slope_3to12m
121,Onset: Limb,NaN,NaN,Yes,0.0,52.0,Female,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.058824
1009,Onset: Other,-324.0,-63.0,Yes,0.0,51.0,Male,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1036,Onset: Bulbar,NaN,NaN,NaN,NaN,67.0,Female,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# leakage free preprocessing

In [4]:
# ==== BUILD y (3→12m ALSFRS slope per ~30d), aligned to features_df ====

import numpy as np, pandas as pd

# --- locate long ALSFRS table ---
def _first_existing(names):
    g = globals()
    for n in names:
        if n in g and isinstance(g[n], pd.DataFrame):
            return g[n], n
    return None, None

als_long, _als_name = _first_existing([
    "alsfrs_long", "alsfrs_all", "alsfrs_full", "alsfrs", "als_long", "alsfrs_df"
])
if als_long is None:
    raise RuntimeError("Long ALSFRS DataFrame not found. Load it (e.g., alsfrs_long).")

# --- detect columns (subject, time, score) ---
def _pick_col(df, cands):
    for c in cands:
        if c in df.columns: return c
    lc = {c.lower(): c for c in df.columns}
    for c in cands:
        if c.lower() in lc: return lc[c.lower()]
    return None

SUBJ  = _pick_col(als_long, ["subject_id","Subject_ID","RID","patient_id"])
TIME  = _pick_col(als_long, ["ALSFRS_Delta","days","Days","days_since_first","days_since_baseline"])
SCORE = _pick_col(als_long, ["ALSFRS_Total_orig","ALSFRS_Total","ALSFRS","ALSFRS_R","ALSFRS_R_Total"])
if any(v is None for v in [SUBJ,TIME,SCORE]):
    raise RuntimeError(f"Could not auto-detect columns: SUBJ={SUBJ}, TIME={TIME}, SCORE={SCORE}")

# --- slope per subject using points in (90, 365] days; per-30-days units ---
def _slope_per30d(g: pd.DataFrame, t_col: str, y_col: str) -> float | None:
    g = g[[t_col, y_col]].dropna()
    g = g[(g[t_col] > 90) & (g[t_col] <= 365)]
    if len(g) < 2:
        return None
    t = g[t_col].to_numpy(dtype=float)
    y = g[y_col].to_numpy(dtype=float)
    a, b = np.polyfit(t, y, deg=1)   # points/day
    return float(a * 30.0)           # ≈ per month

slopes = (
    als_long
    .groupby(SUBJ, group_keys=False)
    .apply(lambda df: _slope_per30d(df, TIME, SCORE))
    .rename("slope_3to12m")
)

# --- align to features_df index; drop subjects without slope ---
if "features_df" not in globals():
    raise RuntimeError("features_df not found. Build your subject-level feature table first.")

# FIXED: Remove target column from features_df if it exists
if 'ALSFRS_slope_3to12m' in features_df.columns:
    features_df = features_df.drop(columns=['ALSFRS_slope_3to12m'])

y = slopes.reindex(features_df.index)
mask = y.notna()
features_df = features_df.loc[mask].copy()
y = y.loc[mask].copy()

print("Built target `y`.")
print("features_df:", features_df.shape, "| y:", y.shape, "| mean:", round(y.mean(),3), "std:", round(y.std(),3))
print(f"✓ Confirmed: Target column removed from features_df")


Built target `y`.
features_df: (2424, 517) | y: (2424,) | mean: -0.383 std: 0.522
✓ Confirmed: Target column removed from features_df


In [5]:
# ==== CLEAN TRAIN/TEST SPLIT (no transforms; no leakage) ====
from sklearn.model_selection import train_test_split
import pandas as pd, numpy as np

# FIXED: Ensure target is not in features
if 'ALSFRS_slope_3to12m' in features_df.columns:
    features_df = features_df.drop(columns=['ALSFRS_slope_3to12m'])

assert all(features_df.index == y.index), "Index mismatch between features and target!"
assert 'ALSFRS_slope_3to12m' not in features_df.columns, "Target column still in features!"

def stratify_bins(y_series, n_bins=10):
    q = pd.qcut(y_series, q=np.minimum(n_bins, max(2, y_series.nunique())), duplicates='drop')
    return pd.factorize(q, sort=True)[0]

bins = stratify_bins(y, n_bins=10)
X_train, X_test, y_train, y_test = train_test_split(
    features_df, y, test_size=0.2, random_state=42, stratify=bins
)
X_train = X_train.copy(); X_test = X_test.copy()
y_train = y_train.copy(); y_test = y_test.copy()

print("✓ Train/test split complete")
print(f"  X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"  y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"  ✓ No target leakage - verified!")


✓ Train/test split complete
  X_train: (1939, 517), X_test: (485, 517)
  y_train: (1939,), y_test: (485,)
  ✓ No target leakage - verified!


In [9]:
# ==== BULLETPROOF PREPROCESSOR (categorical-safe, no leakage) ====
import re
import numpy as np, pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression


class Preprocessor:
    def __init__(self, top_k=25, max_missing=0.5, use_pca=False, pca_components=12,
                 max_cats_per_col=8, numeric_threshold=1.0, force_cat=None):
        """
        numeric_threshold=1.0  -> only columns 100% numeric-convertible are treated as numeric.
        force_cat: list[str] of columns to always treat as categorical (optional).
        """
        self.top_k = top_k
        self.max_missing = max_missing
        self.use_pca = use_pca
        self.pca_components = pca_components
        self.max_cats_per_col = max_cats_per_col
        self.numeric_threshold = numeric_threshold
        self.force_cat = set(force_cat or [])

        # learned
        self.num_cols_, self.cat_cols_ = [], []
        self.cat_maps_ = {}      # col -> kept categories
        self.keep_cols_ = []
        self.scaler_ = None
        self.pca_ = None
        self.num_medians_ = None

    # ----- helpers -----
    @staticmethod
    def _has_letters(sample_values) -> bool:
        # Detect alpha characters in sample of values (flags columns like "Onset: Limb")
        for v in sample_values:
            if pd.isna(v):
                continue
            s = str(v)
            if re.search(r"[A-Za-z]", s):
                return True
        return False

    def _split_num_cat(self, X: pd.DataFrame):
        num_cols, cat_cols = [], []
        for c in X.columns:
            if c in self.force_cat:
                cat_cols.append(c); continue
            s = X[c]
            # quick letter check on 100 non-null samples
            nonnull = s.dropna()
            sample = nonnull.sample(min(100, len(nonnull)), random_state=42) if len(nonnull) else nonnull
            if self._has_letters(sample.values):
                cat_cols.append(c); continue
            # numeric convertibility
            s_num = pd.to_numeric(s, errors="coerce")
            frac_numeric = s_num.notna().mean()
            if frac_numeric >= self.numeric_threshold:
                num_cols.append(c)
            else:
                cat_cols.append(c)
        return num_cols, cat_cols

    def _encode_cats_fit(self, X_cat: pd.DataFrame) -> pd.DataFrame:
        oh = []
        for c in X_cat.columns:
            s = X_cat[c].astype("object")
            s = s.astype(str).where(~s.isna(), "MISSING")
            vc = s.value_counts(dropna=False)
            keep = vc.index.tolist()[: max(1, self.max_cats_per_col - 1)]
            if "MISSING" in s.values and "MISSING" not in keep:
                if len(keep) >= self.max_cats_per_col:
                    keep = keep[:-1] + ["MISSING"]
                else:
                    keep = keep + ["MISSING"]
            keep = list(dict.fromkeys(keep))
            self.cat_maps_[c] = keep
            for k in keep:
                col = f"{c}__{k}"
                oh.append(pd.Series((s == k).astype(np.float32), index=s.index, name=col))
            # OTHER bucket
            other = ~s.isin(keep)
            oh.append(pd.Series(other.astype(np.float32), index=s.index, name=f"{c}__OTHER"))
        return pd.concat(oh, axis=1) if len(oh) else pd.DataFrame(index=X_cat.index)

    def _encode_cats_apply(self, X_cat: pd.DataFrame) -> pd.DataFrame:
        oh = []
        for c in self.cat_cols_:
            s = X_cat[c] if c in X_cat.columns else pd.Series(index=X_cat.index, dtype="object")
            s = s.astype(str).where(~s.isna(), "MISSING")
            keep = self.cat_maps_.get(c, [])
            for k in keep:
                col = f"{c}__{k}"
                oh.append(pd.Series((s == k).astype(np.float32), index=s.index, name=col))
            other = ~s.isin(keep)
            oh.append(pd.Series(other.astype(np.float32), index=s.index, name=f"{c}__OTHER"))
        return pd.concat(oh, axis=1) if len(oh) else pd.DataFrame(index=X_cat.index)

    def _feature_scores(self, X: pd.DataFrame, y: pd.Series) -> pd.Series:
        # Ensure y is pure numpy
        y_np = np.array(y.values if hasattr(y, 'values') else y, dtype=np.float64)

        # RF importance
        rf = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
        rf.fit(X.values, y_np)  # Use .values to ensure numpy array
        s_rf = pd.Series(rf.feature_importances_, index=X.columns)

        # Mutual information
        mi = mutual_info_regression(X.values, y_np, random_state=42)
        s_mi = pd.Series(mi, index=X.columns)

        # |Pearson r|
        def safe_corr(col):
            v = col.values
            if np.std(v) == 0: return 0.0
            return float(abs(np.corrcoef(v, y_np)[0,1]))
        s_pr = X.apply(safe_corr)

        # Blend (normalized)
        def nz_norm(s):
            s = s.fillna(0.0); m = s.max()
            return s / m if m > 0 else s
        blended = 0.5*nz_norm(s_rf) + 0.3*nz_norm(s_mi) + 0.2*nz_norm(s_pr)
        return blended.sort_values(ascending=False)

    # ----- API -----
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        # split by robust content check (no strings into numeric)
        self.num_cols_, self.cat_cols_ = self._split_num_cat(X_train)

        # numeric missingness filter on TRAIN only
        num_keep = []
        if self.num_cols_:
            coerced = X_train[self.num_cols_].apply(pd.to_numeric, errors="coerce")
            miss = coerced.isna().mean()
            num_keep = miss[miss <= self.max_missing].index.tolist()

        cat_keep = self.cat_cols_
        X_tr = X_train[num_keep + cat_keep].copy()

        # numeric block (hard coerce to float32) + train medians
        if num_keep:
            X_tr_num = X_tr[num_keep].apply(pd.to_numeric, errors="coerce").astype(np.float32)
            self.num_medians_ = X_tr_num.median()
            X_tr_num = X_tr_num.fillna(self.num_medians_)
        else:
            X_tr_num = pd.DataFrame(index=X_tr.index, dtype=np.float32)
            self.num_medians_ = pd.Series(dtype=np.float32)

        # categorical block → one-hot (fit)
        X_tr_cat = X_tr[cat_keep] if cat_keep else pd.DataFrame(index=X_tr.index)
        X_tr_cat_oh = self._encode_cats_fit(X_tr_cat)

        # combine
        X_tr_full = pd.concat([X_tr_num, X_tr_cat_oh], axis=1)

        # feature scoring/selection on TRAIN only
        scores = self._feature_scores(X_tr_full, y_train)
        self.keep_cols_ = scores.head(self.top_k).index.tolist()

        # scale fit on TRAIN selected
        self.scaler_ = RobustScaler()
        X_sel = X_tr_full[self.keep_cols_].values  # Use .values for pure numpy
        X_scl = self.scaler_.fit_transform(X_sel)

        # optional PCA
        if self.use_pca:
            n_comp = min(self.pca_components, X_scl.shape[1])
            self.pca_ = PCA(n_components=n_comp, random_state=42)
            self.pca_.fit(X_scl)
        else:
            self.pca_ = None
        return self

    def transform(self, X: pd.DataFrame) -> np.ndarray:
        # numeric (coerce to float32, fill with TRAIN medians)
        if len(self.num_cols_):
            cols_num = [c for c in self.num_cols_ if c in X.columns]
            X_num = X[cols_num].apply(pd.to_numeric, errors="coerce").astype(np.float32)
            # make sure all expected numeric cols exist
            for c in self.num_medians_.index:
                if c not in X_num.columns:
                    X_num[c] = np.nan
            X_num = X_num[self.num_medians_.index]
            X_num = X_num.fillna(self.num_medians_)
        else:
            X_num = pd.DataFrame(index=X.index, dtype=np.float32)

        # categorical
        cols_cat = [c for c in self.cat_cols_ if c in X.columns]
        X_cat = X[cols_cat] if cols_cat else pd.DataFrame(index=X.index)
        X_cat_oh = self._encode_cats_apply(X_cat)

        # combine & align to kept features
        X_full = pd.concat([X_num, X_cat_oh], axis=1)
        for c in self.keep_cols_:
            if c not in X_full.columns:
                X_full[c] = 0.0
        X_full = X_full[self.keep_cols_]

        # CRITICAL: Convert to pure numpy BEFORE scaling
        X_np = X_full.values.astype(np.float32)
        X_scl = self.scaler_.transform(X_np)

        if self.pca_ is not None:
            X_scl = self.pca_.transform(X_scl)

        # CRITICAL: Return pure numpy array with explicit copy
        return np.array(X_scl, dtype=np.float32, copy=True)


In [13]:
# ==== QNN TRAINING - ABSOLUTE FINAL WORKING VERSION ====

import math, time, gc
import numpy as np, pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# ---------------- metrics ----------------
def compute_metrics(y_true, y_pred):
    """Completely tensor-safe metric computation"""
    y_true = np.array(y_true, dtype=np.float64, copy=True).reshape(-1)
    y_pred = np.array(y_pred, dtype=np.float64, copy=True).reshape(-1)

    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae  = float(mean_absolute_error(y_true, y_pred))
    r2   = float(r2_score(y_true, y_pred))
    pcc  = float(pearsonr(y_true, y_pred)[0]) if (np.std(y_true)>0 and np.std(y_pred)>0) else float("nan")
    return rmse, mae, r2, pcc

# --------------- dataset -----------------
class NPDataset(Dataset):
    def __init__(self, X_np, y_np):
        if isinstance(X_np, pd.DataFrame):
            X_np = X_np.values
        if isinstance(y_np, pd.Series):
            y_np = y_np.values

        X_np = np.array(X_np, dtype=np.float32, copy=True)
        y_np = np.array(y_np, dtype=np.float32, copy=True).reshape(-1, 1)

        self.X = torch.from_numpy(X_np)
        self.y = torch.from_numpy(y_np)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --------------- QNN model ----------------
class QNNRegressor(nn.Module):
    def __init__(self, input_dim: int, n_wires: int = 8, n_layers: int = 2, hidden: int = 64):
        super().__init__()
        self.input_dim = input_dim
        self.n_wires = int(min(n_wires, max(1, input_dim)))
        self.n_layers = n_layers

        # Use default.qubit with shots=None for exact simulation
        dev = qml.device("default.qubit", wires=self.n_wires)

        @qml.qnode(dev, interface="torch")  # No diff_method specified - let PennyLane choose
        def qnode(inputs, weights):
            # Use only the first n_wires features
            inputs_use = inputs[..., :self.n_wires]

            qml.AngleEmbedding(inputs_use, wires=range(self.n_wires), rotation="Y")
            qml.StronglyEntanglingLayers(weights, wires=range(self.n_wires))

            # Return list of expectations
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_wires)]

        weight_shapes = {"weights": (self.n_layers, self.n_wires, 3)}
        self.q_layer = qml.qnn.TorchLayer(qnode, weight_shapes)

        self.head = nn.Sequential(
            nn.Linear(self.n_wires + self.input_dim, hidden), nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(hidden, 1)
        )

    def forward(self, x):
        # Get quantum layer output
        q_out = self.q_layer(x)

        # Handle different output shapes from PennyLane
        if isinstance(q_out, (list, tuple)):
            q_out = torch.stack(q_out, dim=-1)
        if len(q_out.shape) == 1:
            q_out = q_out.unsqueeze(0)
        if q_out.shape[-1] != self.n_wires:
            q_out = q_out.reshape(-1, self.n_wires)

        # Concatenate quantum output with classical features
        z = torch.cat([q_out, x], dim=1)
        return self.head(z)

# ----------- train utils -------------
def cosine_warmup_lr(epoch, cfg):
    if epoch < cfg['warmup_epochs']:
        return cfg['lr_start'] + (cfg['lr_max'] - cfg['lr_start']) * (epoch / cfg['warmup_epochs'])
    progress = (epoch - cfg['warmup_epochs']) / max(1, (cfg['epochs'] - cfg['warmup_epochs']))
    return cfg['lr_max'] * 0.5 * (1 + math.cos(math.pi * progress))

def corr_loss(y_hat, y):
    """Correlation loss function"""
    x = y_hat.view(-1)
    t = y.view(-1)
    x_centered = x - x.mean()
    t_centered = t - t.mean()
    numerator = (x_centered * t_centered).mean()
    denominator = torch.sqrt((x_centered ** 2).mean() * (t_centered ** 2).mean()) + 1e-8
    corr = numerator / denominator
    return 1.0 - corr

# ------------- main runner ---------------
def run_qnn_training(
    X_train_df, X_test_df, y_train_s, y_test_s,
    cfg=None,
    pre_top_k=25, use_pca=False, pca_components=12,
    force_cat_cols=None
):
    """Train strictly on train/val; report test once."""
    cfg = cfg or {
        "epochs": 200, "patience": 25,
        "batch": 64, "lr_start": 1e-5, "lr_max": 1e-3, "warmup_epochs": 10,
        "n_wires": 8, "n_layers": 2, "hidden": 64, "corr_lambda": 0.10,
        "val_size": 0.2, "random_state": 42
    }

    print("="*60)
    print("QNN TRAINING (leak-free)")
    print("="*60)

    # Inner train/val split
    X_tr_df, X_val_df, y_tr, y_val = train_test_split(
        X_train_df, y_train_s, test_size=cfg["val_size"],
        random_state=cfg["random_state"], stratify=None
    )

    # Fit preprocessor
    print("\n🔧 Fitting preprocessor...")
    prep = Preprocessor(
        top_k=pre_top_k, use_pca=use_pca, pca_components=pca_components,
        numeric_threshold=1.0, force_cat=force_cat_cols or []
    ).fit(X_tr_df, y_tr)

    # Transform data
    print("🔄 Transforming data...")
    X_tr_np  = prep.transform(X_tr_df)
    X_val_np = prep.transform(X_val_df)
    X_te_np  = prep.transform(X_test_df)

    y_tr_np  = np.array(y_tr.values if hasattr(y_tr, 'values') else y_tr, dtype=np.float32, copy=True)
    y_val_np = np.array(y_val.values if hasattr(y_val, 'values') else y_val, dtype=np.float32, copy=True)
    y_te_np  = np.array(y_test_s.values if hasattr(y_test_s, 'values') else y_test_s, dtype=np.float32, copy=True)

    print(f"✓ Data shapes: X_train={X_tr_np.shape}, X_val={X_val_np.shape}, X_test={X_te_np.shape}")

    # Setup device and loaders
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"✓ Using device: {device}")

    train_loader = DataLoader(NPDataset(X_tr_np, y_tr_np), batch_size=cfg["batch"], shuffle=True, drop_last=False)
    val_loader   = DataLoader(NPDataset(X_val_np, y_val_np), batch_size=cfg["batch"], shuffle=False, drop_last=False)
    test_loader  = DataLoader(NPDataset(X_te_np, y_te_np),   batch_size=cfg["batch"], shuffle=False, drop_last=False)

    # Build model
    print(f"\n🧠 Building QNN model (input_dim={X_tr_np.shape[1]}, n_wires={cfg['n_wires']}, n_layers={cfg['n_layers']})...")
    model = QNNRegressor(input_dim=X_tr_np.shape[1], n_wires=cfg["n_wires"], n_layers=cfg["n_layers"], hidden=cfg["hidden"]).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=cfg["lr_start"])
    mse = nn.MSELoss()

    # Training state
    best = {"val_rmse": float("inf"), "state": None, "val_pred": None}
    patience = 0
    history = {"train_loss": [], "val_rmse": [], "val_pcc": [], "lrs": []}

    print("\n🚀 Starting training...\n")

    for epoch in range(cfg["epochs"]):
        # Adjust learning rate
        lr = cosine_warmup_lr(epoch, cfg)
        for g in opt.param_groups:
            g["lr"] = lr

        # Training
        model.train()
        epoch_loss = 0.0
        n_batches = 0

        try:
            for xb, yb in train_loader:
                xb = xb.to(device)
                yb = yb.to(device)

                opt.zero_grad(set_to_none=True)

                # Forward pass
                pred = model(xb)

                # Compute combined loss
                loss_mse = mse(pred, yb)
                loss_corr = corr_loss(pred, yb)
                loss = loss_mse + cfg["corr_lambda"] * loss_corr

                # Backward pass
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                opt.step()

                # Accumulate loss (detach to prevent memory buildup)
                epoch_loss += loss.detach().item()
                n_batches += 1

        except RuntimeError as e:
            print(f"\n❌ Error during training epoch {epoch+1}: {str(e)}")
            print("This might be due to PennyLane/PyTorch compatibility issues.")
            raise

        avg_loss = epoch_loss / max(1, n_batches)
        history["lrs"].append(lr)
        history["train_loss"].append(avg_loss)

        # Validation
        model.eval()
        vpred_list, vtrue_list = [], []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(device)
                yp = model(xb)

                # Safe conversion to numpy
                yp_np = yp.cpu().numpy().copy().reshape(-1)
                yt_np = yb.cpu().numpy().copy().reshape(-1)

                # Convert to Python floats
                vpred_list.extend([float(v) for v in yp_np])
                vtrue_list.extend([float(v) for v in yt_np])

        # Build pure numpy arrays
        vpred = np.array(vpred_list, dtype=np.float64)
        vtrue = np.array(vtrue_list, dtype=np.float64)
        vrmse, vmae, vr2, vpcc = compute_metrics(vtrue, vpred)
        history["val_rmse"].append(vrmse)
        history["val_pcc"].append(vpcc)

        # Print progress
        if (epoch+1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:3d}/{cfg['epochs']} | TrainLoss {avg_loss:.4f} | "
                  f"Val RMSE {vrmse:.4f} | Val PCC {vpcc:.4f} | LR {lr:.6f}")

        # Early stopping check
        improved = vrmse < best["val_rmse"] - 1e-5
        if improved:
            best["val_rmse"] = vrmse
            best["state"] = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}
            best["val_pred"] = (vtrue.copy(), vpred.copy())
            patience = 0
        else:
            patience += 1
            if patience >= cfg["patience"]:
                print(f"\n⏹️  Early stopping at epoch {epoch+1}")
                break

    # Load best model
    model.load_state_dict(best["state"])
    model = model.to(device).eval()

    # Test evaluation
    print("\n📊 Evaluating on test set...")
    tpred_list, ttrue_list = [], []

    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            yp = model(xb)

            yp_np = yp.cpu().numpy().copy().reshape(-1)
            yt_np = yb.cpu().numpy().copy().reshape(-1)

            tpred_list.extend([float(v) for v in yp_np])
            ttrue_list.extend([float(v) for v in yt_np])

    tpred = np.array(tpred_list, dtype=np.float64)
    ttrue = np.array(ttrue_list, dtype=np.float64)
    rmse_pre, mae_pre, r2_pre, pcc_pre = compute_metrics(ttrue, tpred)

    # Calibration
    vtrue, vpred = best["val_pred"]
    A = np.vstack([vpred, np.ones_like(vpred)]).T
    alpha, beta = np.linalg.lstsq(A, vtrue, rcond=None)[0]
    tpred_cal = alpha * tpred + beta

    rmse_post, mae_post, r2_post, pcc_post = compute_metrics(ttrue, tpred_cal)

    vrmse_pre, _, _, vpcc_pre = compute_metrics(vtrue, vpred)
    vpred_cal = alpha * vpred + beta
    vrmse_post, _, _, vpcc_post = compute_metrics(vtrue, vpred_cal)
    use_cal = (vpcc_post > vpcc_pre) and (vrmse_post <= vrmse_pre + 0.005)

    final = {
        "pre":  {"rmse": rmse_pre,  "mae": mae_pre,  "r2": r2_pre,  "pcc": pcc_pre,  "pred": tpred,     "true": ttrue},
        "post": {"rmse": rmse_post, "mae": mae_post, "r2": r2_post, "pcc": pcc_post, "pred": tpred_cal, "true": ttrue},
        "use_calibrated": use_cal,
        "val_pre":  {"rmse": vrmse_pre,  "pcc": vpcc_pre},
        "val_post": {"rmse": vrmse_post, "pcc": vpcc_post},
        "history": history,
        "prep": prep,
        "cfg": cfg
    }

    choice = "Calibrated" if use_cal else "Raw"
    chosen = final["post"] if use_cal else final["pre"]
    print("\n" + "="*60)
    print(f"🎯 QNN TEST — Using: {choice}")
    print(f"RMSE: {chosen['rmse']:.4f} | MAE: {chosen['mae']:.4f} | R²: {chosen['r2']:.4f} | PCC: {chosen['pcc']:.4f}")
    print("="*60)
    return final

# ---- config & run ----
cfg = {
    "epochs": 200, "patience": 25, "batch": 64,
    "lr_start": 1e-5, "lr_max": 1e-3, "warmup_epochs": 10,
    "n_wires": 8, "n_layers": 2, "hidden": 64,
    "corr_lambda": 0.10, "val_size": 0.2, "random_state": 42
}

force_cat_cols = [c for c in X_train.columns if "onset" in c.lower() or "type" in c.lower()]

artifacts_qnn = run_qnn_training(
    X_train_df=X_train, X_test_df=X_test,
    y_train_s=y_train, y_test_s=y_test,
    cfg=cfg, pre_top_k=25, use_pca=False, pca_components=12,
    force_cat_cols=force_cat_cols
)


QNN TRAINING (leak-free)

🔧 Fitting preprocessor...
🔄 Transforming data...
✓ Data shapes: X_train=(1551, 25), X_val=(388, 25), X_test=(485, 25)
✓ Using device: cpu

🧠 Building QNN model (input_dim=25, n_wires=8, n_layers=2)...

🚀 Starting training...

Epoch   1/200 | TrainLoss 0.5234 | Val RMSE 0.5718 | Val PCC -0.3305 | LR 0.000010
Epoch   5/200 | TrainLoss 0.2908 | Val RMSE 0.4489 | Val PCC 0.4406 | LR 0.000406
Epoch  10/200 | TrainLoss 0.2440 | Val RMSE 0.4309 | Val PCC 0.4850 | LR 0.000901
Epoch  15/200 | TrainLoss 0.2450 | Val RMSE 0.4289 | Val PCC 0.4974 | LR 0.000999
Epoch  20/200 | TrainLoss 0.2331 | Val RMSE 0.4248 | Val PCC 0.5066 | LR 0.000994
Epoch  25/200 | TrainLoss 0.2382 | Val RMSE 0.4237 | Val PCC 0.5084 | LR 0.000987
Epoch  30/200 | TrainLoss 0.2427 | Val RMSE 0.4247 | Val PCC 0.5095 | LR 0.000976
Epoch  35/200 | TrainLoss 0.2316 | Val RMSE 0.4240 | Val PCC 0.5123 | LR 0.000961
Epoch  40/200 | TrainLoss 0.2318 | Val RMSE 0.4226 | Val PCC 0.5121 | LR 0.000944
Epoch  45

# Step 2 Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression, VarianceThreshold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
import gc
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("STEP 2: BREAKTHROUGH PREPROCESSING")
print("=" * 60)

# -------------------------------
# 1. Filter valid samples
# -------------------------------
print("\n🎯 Filtering samples with valid target...")
initial_count = len(features_df)
features_df = features_df[features_df['ALSFRS_slope_3to12m'].notna()].copy()
print(f"✅ Removed {initial_count - len(features_df)} samples without target")
print(f"✅ Remaining samples: {len(features_df):,}")

# Separate features and target (ORIGINAL SCALE!)
y = features_df['ALSFRS_slope_3to12m'].copy()
X = features_df.drop('ALSFRS_slope_3to12m', axis=1)

print(f"\n📊 Target statistics (ORIGINAL SCALE):")
print(f"  - Mean: {y.mean():.4f}")
print(f"  - Std: {y.std():.4f}")
print(f"  - Range: [{y.min():.4f}, {y.max():.4f}]")

# -------------------------------
# 2. Handle categorical variables
# -------------------------------
print("\n🏷️  Encoding categorical variables...")
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"✅ Found {len(categorical_cols)} categorical columns: {categorical_cols}")

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = X[col].fillna('Missing')
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# -------------------------------
# 3. Quick missing value handling
# -------------------------------
print("\n🔧 Handling missing values...")

# Drop >50% missing
missing_pct = X.isnull().sum() / len(X)
cols_to_drop = missing_pct[missing_pct > 0.5].index.tolist()
print(f"  📉 Dropping {len(cols_to_drop)} columns with >50% missing")
X = X.drop(columns=cols_to_drop)

# Simple median imputation (fast, memory-efficient)
print(f"  🔹 Applying median imputation...")
for col in X.columns:
    if X[col].isnull().any():
        X[col].fillna(X[col].median(), inplace=True)

X = X.fillna(0)
print(f"✅ Missing value handling complete. Shape: {X.shape}")

# -------------------------------
# 4. Create critical interaction features
# -------------------------------
print("\n🔀 Creating interaction features...")
interaction_pairs = [
    ('Q5_Cutting_mean', 'Weight_median'),
    ('FVC_mean', 'Respiratory_Rate_mean'),
    ('Q1_Speech_mean', 'Q3_Swallowing_mean'),
]

created = 0
for col1, col2 in interaction_pairs:
    if col1 in X.columns and col2 in X.columns:
        X[f'{col1}_x_{col2}'] = X[col1] * X[col2]
        created += 1
print(f"✅ Created {created} interaction features")

# -------------------------------
# 5. Variance threshold
# -------------------------------
print("\n📊 Removing low-variance features...")
var_selector = VarianceThreshold(threshold=0.01)
X = X[X.columns[var_selector.fit(X).get_support()]]
print(f"✅ After variance threshold: {X.shape[1]} features")

# -------------------------------
# 6. Multi-method feature selection (Top 25)
# -------------------------------
print("\n🎯 Multi-method feature selection (Top 25)...")

# Method 1: Random Forest (50% weight)
print("  🌲 Random Forest importance...")
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X, y)
rf_importance = pd.Series(rf.feature_importances_, index=X.columns)

# Method 2: Mutual Information (30% weight)
print("  📊 Mutual Information...")
mi_scores = mutual_info_regression(X, y, random_state=42, n_neighbors=5)
mi_importance = pd.Series(mi_scores, index=X.columns)

# Method 3: Pearson Correlation (20% weight)
print("  📈 Pearson correlation...")
corr_importance = X.corrwith(y).abs()

# Weighted ensemble ranking
rank_rf = rf_importance.rank(ascending=False)
rank_mi = mi_importance.rank(ascending=False)
rank_corr = corr_importance.rank(ascending=False)

combined_rank = (0.5 * rank_rf + 0.3 * rank_mi + 0.2 * rank_corr)
combined_rank = combined_rank.sort_values()

# Select top 25 features
n_features_select = 25
top_25_features = combined_rank.head(n_features_select).index.tolist()

print(f"\n  ✅ Selected top {n_features_select} features")
print(f"  🏆 Top 10 features:")
for i, feat in enumerate(top_25_features[:10], 1):
    print(f"     {i:2d}. {feat}")

X_selected = X[top_25_features].copy()

# Clear memory
del X, rf
gc.collect()

# -------------------------------
# 7. Robust scaling for features ONLY
# -------------------------------
print("\n⚖️  Applying Robust Scaling to features...")
scaler = RobustScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X_selected),
    columns=X_selected.columns,
    index=X_selected.index
)

print(f"✅ Features scaled. Shape: {X_scaled.shape}")
print(f"   Mean range: [{X_scaled.mean().min():.4f}, {X_scaled.mean().max():.4f}]")

# -------------------------------
# 8. Train-test split (NO TARGET SCALING!)
# -------------------------------
print("\n✂️  Creating train-test split...")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

print(f"✅ Train set: {X_train.shape[0]} samples")
print(f"✅ Test set: {X_test.shape[0]} samples")
print(f"✅ Features: {X_train.shape[1]}")

# -------------------------------
# 9. Save preprocessing artifacts
# -------------------------------
preprocessing_artifacts = {
    'scaler': scaler,
    'selected_features': top_25_features,
    'label_encoders': label_encoders
}

print("\n" + "=" * 60)
print("✅ STEP 2 COMPLETED")
print("=" * 60)
print(f"\n📊 Preprocessing Summary:")
print(f"  - Training samples: {len(X_train):,}")
print(f"  - Test samples: {len(X_test):,}")
print(f"  - Selected features: {n_features_select}")
print(f"  - Target: ORIGINAL SCALE (no scaling!)")
print(f"  - Target mean: {y.mean():.4f}, std: {y.std():.4f}")
print(f"  - Target range: [{y.min():.4f}, {y.max():.4f}]")

print("\n🔍 Feature preview:")
print(X_train.head(3))
print("\n🎯 Target preview:")
print(y_train.head(10).values)

STEP 2: BREAKTHROUGH PREPROCESSING

🎯 Filtering samples with valid target...
✅ Removed 3 samples without target
✅ Remaining samples: 2,439

📊 Target statistics (ORIGINAL SCALE):
  - Mean: -0.3881
  - Std: 0.4965
  - Range: [-3.1000, 1.0526]

🏷️  Encoding categorical variables...
✅ Found 3 categorical columns: ['Site_of_Onset', 'Subject_used_Riluzole', 'Sex']

🔧 Handling missing values...
  📉 Dropping 295 columns with >50% missing
  🔹 Applying median imputation...
✅ Missing value handling complete. Shape: (2439, 222)

🔀 Creating interaction features...
✅ Created 2 interaction features

📊 Removing low-variance features...
✅ After variance threshold: 223 features

🎯 Multi-method feature selection (Top 25)...
  🌲 Random Forest importance...
  📊 Mutual Information...
  📈 Pearson correlation...

  ✅ Selected top 25 features
  🏆 Top 10 features:
      1. ALSFRS_Total_orig_first
      2. ALSFRS_Total_orig_q25
      3. ALSFRS_Total_orig_last
      4. ALSFRS_Total_orig_max
      5. ALSFRS_Total_

# Qnn Model

In [ ]:
import pennylane as qml
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA
import gc

print("=" * 60)
print("STEP 3: BREAKTHROUGH QNN MODEL")
print("=" * 60)

# -------------------------------
# 1. Configuration (Optimized for breakthrough)
# -------------------------------
print("\n⚙️  Breakthrough Configuration...")

config = {
    'n_features': X_train.shape[1],  # 25 features
    'n_qubits': 12,  # More than 10, less RAM than 18
    'n_layers': 4,   # Deeper than 3 for more expressiveness
    'batch_size': 24,  # Balance between speed and memory
    'learning_rate_start': 0.0001,  # Very low start for warmup
    'learning_rate_max': 0.003,     # Peak learning rate
    'n_epochs': 100,
    'warmup_epochs': 10,
    'early_stopping_patience': 20,
}

print(f"✅ Configuration:")
for key, value in config.items():
    print(f"   {key}: {value}")

# -------------------------------
# 2. PCA with Better Variance Retention
# -------------------------------
print(f"\n🔬 Applying PCA: {config['n_features']} → {config['n_qubits']} dimensions")

pca = PCA(n_components=config['n_qubits'], random_state=42)
X_train_qnn = pca.fit_transform(X_train)
X_test_qnn = pca.transform(X_test)

explained_var = pca.explained_variance_ratio_.sum()
print(f"✅ Explained variance: {explained_var:.1%}")
print(f"✅ Quantum input shape: {X_train_qnn.shape}")

# Add to preprocessing artifacts
preprocessing_artifacts['pca'] = pca

# Clear memory
del X_train, X_test
gc.collect()

# -------------------------------
# 3. Enhanced Quantum Circuit
# -------------------------------
print("\n🔮 Building enhanced quantum circuit...")

dev = qml.device('default.qubit', wires=config['n_qubits'])

@qml.qnode(dev, interface="torch", diff_method="backprop")
def quantum_circuit(inputs, weights):
    """
    Enhanced quantum circuit with:
    - Double rotation encoding
    - 3-parameter rotations (RX, RY, RZ)
    - Ring + Pairwise entanglement
    """
    n_wires = len(inputs)

    # Normalize input
    norm = torch.sqrt(torch.sum(inputs**2)) + 1e-8
    normalized_inputs = inputs / norm

    # Enhanced data encoding (2 rotations per qubit)
    for i in range(n_wires):
        qml.RY(normalized_inputs[i] * np.pi, wires=i)
        qml.RZ(normalized_inputs[i] * np.pi / 2, wires=i)

    # Apply parameterized layers
    for layer_idx in range(weights.shape[0]):
        # 3-parameter rotations for maximum expressiveness
        for i in range(n_wires):
            qml.RX(weights[layer_idx, i, 0], wires=i)
            qml.RY(weights[layer_idx, i, 1], wires=i)
            qml.RZ(weights[layer_idx, i, 2], wires=i)

        # Strong entanglement: Ring topology
        for i in range(n_wires):
            qml.CNOT(wires=[i, (i + 1) % n_wires])

        # Additional pairwise entanglement
        for i in range(0, n_wires - 1, 2):
            qml.CNOT(wires=[i, i + 1])

    # Measure all qubits
    return [qml.expval(qml.PauliZ(i)) for i in range(n_wires)]

print(f"✅ Quantum circuit created:")
print(f"   - Qubits: {config['n_qubits']}")
print(f"   - Layers: {config['n_layers']}")
print(f"   - Encoding: Double rotation (RY + RZ)")
print(f"   - Rotations per layer: 3 (RX, RY, RZ)")
print(f"   - Entanglement: Ring + Pairwise")
print(f"   - Total quantum params: {config['n_layers'] * config['n_qubits'] * 3}")

# -------------------------------
# 4. Hybrid QNN Model with Deeper Classical Head
# -------------------------------
print("\n🧬 Building hybrid quantum-classical model...")

class BreakthroughQNN(nn.Module):
    """Enhanced hybrid model with deeper classical processing"""

    def __init__(self, n_qubits, n_layers):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers

        # Quantum weights (3 parameters per qubit per layer)
        self.q_weights = nn.Parameter(
            torch.randn(n_layers, n_qubits, 3, requires_grad=True) * 0.05
        )

        # Deeper classical post-processing with BatchNorm
        self.fc1 = nn.Linear(n_qubits, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(32, 16)
        self.bn2 = nn.BatchNorm1d(16)
        self.dropout2 = nn.Dropout(0.25)

        self.fc3 = nn.Linear(16, 8)
        self.bn3 = nn.BatchNorm1d(8)
        self.dropout3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(8, 1)

    def forward(self, x):
        batch_size = x.shape[0]

        # Quantum processing
        quantum_outputs = []
        for i in range(batch_size):
            q_out = quantum_circuit(x[i], self.q_weights)
            quantum_outputs.append(torch.stack(q_out))

        quantum_output = torch.stack(quantum_outputs).float()

        # Deep classical post-processing
        x = torch.relu(self.bn1(self.fc1(quantum_output)))
        x = self.dropout1(x)

        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        x = self.fc4(x)

        return x.squeeze()

# Initialize model
model = BreakthroughQNN(
    n_qubits=config['n_qubits'],
    n_layers=config['n_layers']
).float()

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
quantum_params = config['n_layers'] * config['n_qubits'] * 3
classical_params = total_params - quantum_params

print(f"✅ Hybrid QNN model created")
print(f"\n📊 Model Parameters:")
print(f"   - Quantum parameters: {quantum_params:,}")
print(f"   - Classical parameters: {classical_params:,}")
print(f"   - Total trainable parameters: {total_params:,}")

# -------------------------------
# 5. Training Setup
# -------------------------------
print("\n⚙️  Training setup...")

# Huber loss (robust to outliers in ALS data)
criterion = nn.HuberLoss(delta=0.5)
print(f"✅ Loss function: Huber Loss (delta=0.5, robust to outliers)")

# AdamW optimizer with weight decay
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['learning_rate_start'],
    weight_decay=1e-4
)
print(f"✅ Optimizer: AdamW (weight_decay=1e-4)")

# Learning rate scheduler (will be applied in training)
print(f"✅ LR Schedule: Warmup ({config['warmup_epochs']} epochs) + Cosine decay")

# -------------------------------
# 6. Prepare Data Loaders
# -------------------------------
print("\n📦 Preparing data loaders...")

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_qnn)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test_qnn)
y_test_tensor = torch.FloatTensor(y_test.values)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders with moderate batch size
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    drop_last=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config['batch_size'],
    shuffle=False
)

print(f"✅ Train batches: {len(train_loader)}")
print(f"✅ Test batches: {len(test_loader)}")

# -------------------------------
# 7. Test Forward Pass
# -------------------------------
print("\n🧪 Testing model forward pass...")

model.eval()
with torch.no_grad():
    sample_batch = X_train_tensor[:2]
    try:
        output = model(sample_batch)
        print(f"✅ Forward pass successful!")
        print(f"   Input shape: {sample_batch.shape}")
        print(f"   Output shape: {output.shape}")
        print(f"   Output dtype: {output.dtype}")
        print(f"   Sample outputs: {output.numpy()}")
    except Exception as e:
        print(f"❌ Error in forward pass: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "=" * 60)
print("✅ STEP 3 COMPLETED SUCCESSFULLY")
print("=" * 60)
print(f"\n📋 Model Summary:")
print(f"  - Architecture: Hybrid Quantum-Classical")
print(f"  - Quantum: {config['n_qubits']} qubits, {config['n_layers']} layers")
print(f"  - Encoding: Double rotation (RY + RZ)")
print(f"  - Entanglement: Ring + Pairwise")
print(f"  - Classical: 4 layers (32→16→8→1) with BatchNorm & Dropout")
print(f"  - Total parameters: {total_params:,}")
print(f"  - Data: 1,951 train, 488 test")
print(f"  - Ready for training: ✅")

STEP 3: BREAKTHROUGH QNN MODEL

⚙️  Breakthrough Configuration...
✅ Configuration:
   n_features: 25
   n_qubits: 12
   n_layers: 4
   batch_size: 24
   learning_rate_start: 0.0001
   learning_rate_max: 0.003
   n_epochs: 100
   warmup_epochs: 10
   early_stopping_patience: 20

🔬 Applying PCA: 25 → 12 dimensions
✅ Explained variance: 98.8%
✅ Quantum input shape: (1951, 12)

🔮 Building enhanced quantum circuit...
✅ Quantum circuit created:
   - Qubits: 12
   - Layers: 4
   - Encoding: Double rotation (RY + RZ)
   - Rotations per layer: 3 (RX, RY, RZ)
   - Entanglement: Ring + Pairwise
   - Total quantum params: 144

🧬 Building hybrid quantum-classical model...
✅ Hybrid QNN model created

📊 Model Parameters:
   - Quantum parameters: 144
   - Classical parameters: 1,201
   - Total trainable parameters: 1,345

⚙️  Training setup...
✅ Loss function: Huber Loss (delta=0.5, robust to outliers)
✅ Optimizer: AdamW (weight_decay=1e-4)
✅ LR Schedule: Warmup (10 epochs) + Cosine decay

📦 Preparing

# Qnn Training

In [ ]:
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import numpy as np
import torch
import gc

print("=" * 60)
print("STEP 4: BREAKTHROUGH TRAINING")
print("=" * 60)

# -------------------------------
# 1. Learning Rate Scheduler Function
# -------------------------------
def get_lr(epoch, config):
    """Warmup + Cosine annealing learning rate schedule"""
    if epoch < config['warmup_epochs']:
        # Linear warmup
        return config['learning_rate_start'] + \
               (config['learning_rate_max'] - config['learning_rate_start']) * \
               (epoch / config['warmup_epochs'])
    else:
        # Cosine annealing
        progress = (epoch - config['warmup_epochs']) / \
                   (config['n_epochs'] - config['warmup_epochs'])
        return config['learning_rate_max'] * 0.5 * (1 + np.cos(np.pi * progress))

# -------------------------------
# 2. Training and Evaluation Functions
# -------------------------------
def train_epoch(model, loader, criterion, optimizer, epoch, config):
    """Train for one epoch with dynamic learning rate"""
    model.train()
    total_loss = 0

    # Update learning rate
    lr = get_lr(epoch, config)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    for batch_x, batch_y in loader:
        optimizer.zero_grad()
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
        loss.backward()

        # Gradient clipping (tighter for stability)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        optimizer.step()
        total_loss += loss.item()

        # Clear memory
        del pred, loss

    gc.collect()
    return total_loss / len(loader), lr

def evaluate(model, loader):
    """Evaluate model on dataset"""
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_x, batch_y in loader:
            pred = model(batch_x)
            predictions.extend(pred.numpy())
            actuals.extend(batch_y.numpy())
            del pred

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    pcc, _ = pearsonr(actuals, predictions)

    return rmse, mae, r2, pcc, predictions, actuals

# -------------------------------
# 3. Training Loop with Advanced Strategy
# -------------------------------
print("\n🚀 Starting breakthrough training...")
print(f"   Target: RMSE < 0.35, PCC > 0.65")
print(f"   Strategy: Warmup + Cosine decay + Heavy RMSE focus")
print(f"   Epochs: {config['n_epochs']}")
print(f"   Patience: {config['early_stopping_patience']}")
print("-" * 60)

# Tracking
best_score = -float('inf')
best_rmse = float('inf')
best_pcc = -1
best_mae = float('inf')
best_r2 = -1
patience_counter = 0
start_time = time.time()

history = {
    'train_loss': [],
    'test_rmse': [],
    'test_pcc': [],
    'learning_rates': []
}

for epoch in range(config['n_epochs']):
    epoch_start = time.time()

    # Train
    train_loss, lr = train_epoch(model, train_loader, criterion, optimizer, epoch, config)

    # Evaluate
    rmse, mae, r2, pcc, _, _ = evaluate(model, test_loader)

    # Store history
    history['train_loss'].append(train_loss)
    history['test_rmse'].append(rmse)
    history['test_pcc'].append(pcc)
    history['learning_rates'].append(lr)

    # Combined score with HEAVY weight on RMSE (2x)
    # Lower RMSE is better, higher PCC is better
    combined_score = -2.0 * rmse + pcc

    # Track best model
    if combined_score > best_score:
        best_score = combined_score
        best_rmse = rmse
        best_pcc = pcc
        best_mae = mae
        best_r2 = r2
        best_model_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1

    epoch_time = time.time() - epoch_start

    # Print progress
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}/{config['n_epochs']} | "
              f"Loss: {train_loss:.4f} | "
              f"RMSE: {rmse:.4f} | "
              f"MAE: {mae:.4f} | "
              f"PCC: {pcc:.4f} | "
              f"R²: {r2:.4f} | "
              f"LR: {lr:.6f} | "
              f"Time: {epoch_time:.1f}s")

    # Early stopping
    if patience_counter >= config['early_stopping_patience']:
        print(f"\n⏹️  Early stopping triggered at epoch {epoch+1}")
        print(f"   No improvement for {config['early_stopping_patience']} epochs")
        break

total_time = time.time() - start_time
print("-" * 60)
print(f"✅ Training completed in {total_time:.1f}s ({total_time/60:.1f} min)")

# -------------------------------
# 4. Load Best Model and Final Evaluation
# -------------------------------
print("\n📊 Loading best model and final evaluation...")

model.load_state_dict(best_model_state)
final_rmse, final_mae, final_r2, final_pcc, predictions, actuals = evaluate(model, test_loader)

print("\n" + "=" * 60)
print("🏆 BREAKTHROUGH RESULTS")
print("=" * 60)

print(f"\n📊 Final Test Performance:")
print(f"   RMSE: {final_rmse:.4f}")
print(f"   MAE: {final_mae:.4f}")
print(f"   R² Score: {final_r2:.4f}")
print(f"   Pearson Correlation (PCC): {final_pcc:.4f}")

# -------------------------------
# 5. Compare with Targets
# -------------------------------
print(f"\n📈 Performance vs Targets:")
print(f"   Target RMSE: < 0.35")
print(f"   Achieved RMSE: {final_rmse:.4f}")

if final_rmse < 0.35:
    improvement = ((0.58 - final_rmse) / 0.58) * 100
    print(f"   ✅ RMSE TARGET ACHIEVED!")
    print(f"   🎉 {improvement:.1f}% improvement from original (0.58)")
else:
    gap = final_rmse - 0.35
    reduction_from_041 = ((0.41 - final_rmse) / 0.41) * 100
    print(f"   📍 Gap to target: {gap:.4f}")
    print(f"   📉 Reduced from 0.41: {reduction_from_041:.1f}%")

print(f"\n   Target PCC: > 0.65")
print(f"   Achieved PCC: {final_pcc:.4f}")

if final_pcc > 0.65:
    print(f"   ✅ PCC TARGET ACHIEVED!")
else:
    gap = 0.65 - final_pcc
    print(f"   📍 Gap to target: {gap:.4f}")

# -------------------------------
# 6. Training History Analysis
# -------------------------------
print(f"\n📊 Training Statistics:")
print(f"   Total epochs: {len(history['train_loss'])}")
print(f"   Best RMSE: {best_rmse:.4f}")
print(f"   Best PCC: {best_pcc:.4f}")
print(f"   Best MAE: {best_mae:.4f}")
print(f"   Best R²: {best_r2:.4f}")
print(f"   Final LR: {history['learning_rates'][-1]:.6f}")
print(f"   Training time: {total_time/60:.1f} minutes")

# -------------------------------
# 7. Sample Predictions Analysis
# -------------------------------
print("\n🔍 Detailed Prediction Analysis (first 15 samples):")
print(f"{'Actual':>10} | {'Predicted':>10} | {'Error':>10} | {'% Error':>10}")
print("-" * 50)

for i in range(min(15, len(actuals))):
    actual = actuals[i]
    pred = predictions[i]
    error = actual - pred
    pct_error = (error / actual * 100) if actual != 0 else 0
    print(f"{actual:>10.4f} | {pred:>10.4f} | {error:>10.4f} | {pct_error:>9.1f}%")

# Error statistics
errors = actuals - predictions
print(f"\n📊 Error Statistics:")
print(f"   Mean error: {errors.mean():.4f}")
print(f"   Std error: {errors.std():.4f}")
print(f"   Max overestimation: {errors.max():.4f}")
print(f"   Max underestimation: {errors.min():.4f}")
print(f"   Median absolute error: {np.median(np.abs(errors)):.4f}")

# -------------------------------
# 8. Save Model and Artifacts
# -------------------------------
print("\n💾 Saving model and artifacts...")

final_artifacts = {
    'model_state': best_model_state,
    'config': config,
    'preprocessing': preprocessing_artifacts,
    'history': history,
    'best_metrics': {
        'rmse': best_rmse,
        'mae': best_mae,
        'r2': best_r2,
        'pcc': best_pcc
    }
}

print("✅ Artifacts ready for saving")

print("\n" + "=" * 60)
print("✅ STEP 4 COMPLETED SUCCESSFULLY")
print("=" * 60)

print("\n🎉 BREAKTHROUGH TRAINING COMPLETE!")
print(f"\n🎯 Final Summary:")
print(f"   Original Model:  RMSE=0.58, PCC=0.706")
print(f"   Previous Best:   RMSE=0.41, PCC=0.51")
print(f"   This Model:      RMSE={final_rmse:.4f}, PCC={final_pcc:.4f}")

if final_rmse < 0.35 and final_pcc > 0.65:
    print(f"\n🏆 BOTH TARGETS ACHIEVED! 🏆")
elif final_rmse < 0.35:
    print(f"\n✅ RMSE TARGET ACHIEVED!")
    print(f"📍 PCC needs {0.65 - final_pcc:.4f} more improvement")
elif final_pcc > 0.65:
    print(f"\n✅ PCC TARGET ACHIEVED!")
    print(f"📍 RMSE needs {final_rmse - 0.35:.4f} more reduction")
else:
    print(f"\n📍 Close but not yet achieved:")
    print(f"   RMSE gap: {final_rmse - 0.35:.4f}")
    print(f"   PCC gap: {0.65 - final_pcc:.4f}")
    print(f"\n💡 Next steps for improvement:")
    print(f"   - Try ensemble of multiple QNN models")
    print(f"   - Experiment with different quantum encodings")
    print(f"   - Adjust hyperparameters (layers, qubits)")
    print(f"   - Consider data augmentation techniques")

print("\n" + "=" * 60)


STEP 4: BREAKTHROUGH TRAINING

🚀 Starting breakthrough training...
   Target: RMSE < 0.35, PCC > 0.65
   Strategy: Warmup + Cosine decay + Heavy RMSE focus
   Epochs: 100
   Patience: 20
------------------------------------------------------------
Epoch   1/100 | Loss: 0.1545 | RMSE: 0.5604 | MAE: 0.3861 | PCC: 0.1872 | R²: -0.3337 | LR: 0.000100 | Time: 479.4s
Epoch   5/100 | Loss: 0.0830 | RMSE: 0.4295 | MAE: 0.3015 | PCC: 0.4846 | R²: 0.2165 | LR: 0.001260 | Time: 469.5s
Epoch  10/100 | Loss: 0.0754 | RMSE: 0.4173 | MAE: 0.2898 | PCC: 0.5249 | R²: 0.2605 | LR: 0.002710 | Time: 468.9s
Epoch  15/100 | Loss: 0.0727 | RMSE: 0.4168 | MAE: 0.2891 | PCC: 0.5281 | R²: 0.2623 | LR: 0.002985 | Time: 465.0s
Epoch  20/100 | Loss: 0.0719 | RMSE: 0.4100 | MAE: 0.2881 | PCC: 0.5404 | R²: 0.2862 | LR: 0.002927 | Time: 466.7s
Epoch  25/100 | Loss: 0.0714 | RMSE: 0.4088 | MAE: 0.2841 | PCC: 0.5445 | R²: 0.2903 | LR: 0.002824 | Time: 469.8s
Epoch  30/100 | Loss: 0.0698 | RMSE: 0.4130 | MAE: 0.2889 | P

# Classical Model

In [ ]:
"""
Paper-style model implementation for ALSFRS slope prediction (3–12 months)
using THIS NOTEBOOK's preprocessing outputs.

What this script expects to already exist in memory (from your Step 1 & Step 2 cells):
- X_train, X_test, y_train, y_test            # from your preprocessing (original target scale)
- features_df                                  # index = subject_id, includes the target column
- alsfrs_3m                                    # first-90-day ALSFRS rows (subject_id, ALSFRS_Delta, Q* items, etc.)

It will:
1) Build the FFNN baseline on selected summary features (your X_train).
2) Build ALSFRS visit-matrix (n_q x 5) per patient from alsfrs_3m (first 90d) with last-value carry-forward.
3) Build a CNN-fusion model: conv over (n_q × 5) + non‑ALSFRS tabular summaries → FFNN head.
4) Build an RNN-fusion model: 2 recurrent layers over sequence length=5 (features=n_q) + non‑ALSFRS summaries → FFNN head.
5) Train models with early stopping; evaluate RMSD & PCC on X_test; optional bootstrap CIs.
6) Create a simple ensemble (FFNN+CNN average) as in the paper.

Note:
- The paper mentions "recurrent" without specifying cell; here we default to LSTM but you can switch to GRU.
- If your selected features (top_25_features) include ALSFRS-derived summaries, we try to exclude them from the fusion "non‑ALS" branch by name pattern (columns starting with 'Q'). Adjust the predicate if your column names differ.
- Keep target on its ORIGINAL scale (no scaling), as in the paper.
"""

import re
import gc
import math
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# =============================
# Utilities: metrics & seed
# =============================

def set_global_seed(seed: int = 42):
    import random, os
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)


def rmsd(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))


def pcc(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    if y_true.std() == 0 or y_pred.std() == 0:
        return float('nan')
    return float(np.corrcoef(y_true, y_pred)[0, 1])


def bootstrap_ci(y_true: np.ndarray, y_pred: np.ndarray, metric_fn, n_boot: int = 10000, alpha: float = 0.05, seed: int = 42) -> Tuple[float, Tuple[float, float]]:
    """ Nonparametric bootstrap CI for a metric. """
    set_global_seed(seed)
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)
    stats = []
    idx = np.arange(n)
    for _ in range(n_boot):
        bs = np.random.choice(idx, size=n, replace=True)
        stats.append(metric_fn(y_true[bs], y_pred[bs]))
    stats = np.sort(np.array(stats))
    low = stats[int((alpha/2) * n_boot)]
    high = stats[int((1 - alpha/2) * n_boot)]
    point = metric_fn(y_true, y_pred)
    return point, (float(low), float(high))


# ======================================================
# Build ALSFRS visit-matrix M (n_q × max_visits) per id
# ======================================================

def _infer_question_cols(df: pd.DataFrame) -> List[str]:
    """Infer ALSFRS question columns like 'Q1_Speech', 'Q3_Swallowing', ...
    We look for columns that start with Q<digit> and are numeric.
    """
    qcols = []
    for c in df.columns:
        if re.match(r"^Q\d+", str(c)) and pd.api.types.is_numeric_dtype(df[c]):
            qcols.append(c)
    # Sort by the numeric part to maintain order Q1..Q11
    def q_key(s: str):
        m = re.match(r"^Q(\d+)", s)
        return int(m.group(1)) if m else 999
    qcols.sort(key=q_key)
    if not qcols:
        raise ValueError("Could not infer ALSFRS question columns (Q1..). Please check column names in alsfrs_3m.")
    return qcols


def build_alsfrs_matrices(
    alsfrs_3m: pd.DataFrame,
    patient_ids: List[int],
    max_visits: int = 5,
    time_col: str = "ALSFRS_Delta",
    question_cols: List[str] = None,
) -> Tuple[np.ndarray, List[str]]:
    """
    Build ALSFRS matrix A with shape (N, n_q, max_visits, 1) in the same order as patient_ids.
    For each patient: take up to the first `max_visits` within 90 days, sort by time, and
    last-value carry-forward to fill missing visits; remaining NaNs become 0.
    Returns: (A, used_question_cols)
    """
    if question_cols is None:
        question_cols = _infer_question_cols(alsfrs_3m)

    n_q = len(question_cols)
    A = np.zeros((len(patient_ids), n_q, max_visits), dtype=np.float32)

    for i, pid in enumerate(patient_ids):
        g = alsfrs_3m[alsfrs_3m['subject_id'] == pid].copy()
        if time_col not in g.columns:
            raise ValueError(f"'{time_col}' not found in alsfrs_3m")
        g = g.sort_values(time_col)
        # Select first `max_visits` rows
        g = g.iloc[:max_visits]
        # Extract question matrix (rows=visits, cols=questions)
        M = g[question_cols].to_numpy(dtype=np.float32)  # shape (v, n_q)
        # Transpose to (n_q, v)
        M = M.T
        # If fewer than max_visits, pad by last-value carry-forward along time axis
        if M.shape[1] < max_visits:
            # Forward-fill along time dimension
            pad_len = max_visits - M.shape[1]
            if M.shape[1] == 0:
                # no visit rows at all → keep zeros
                M_filled = np.zeros((n_q, max_visits), dtype=np.float32)
            else:
                M_filled = np.concatenate([M, np.repeat(M[:, -1:], pad_len, axis=1)], axis=1)
        else:
            M_filled = M[:, :max_visits]
        # Replace remaining NaNs with 0
        M_filled = np.nan_to_num(M_filled, nan=0.0)
        A[i] = M_filled

    # add channel dim
    A = A[..., np.newaxis]  # (N, n_q, max_visits, 1)
    return A, question_cols


# ==========================================
# Train/Val helpers and model constructors
# ==========================================

def compile_and_fit(model: keras.Model, X_train, y_train, X_val, y_val, epochs=300, batch_size=64, patience=20, verbose=1):
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    cb = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)]
    hist = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=cb,
        verbose=verbose
    )
    return hist


def make_ffnn(input_dim: int) -> keras.Model:
    inp = layers.Input(shape=(input_dim,), name='tabular_in')
    x = layers.Dense(128, activation='relu')(inp)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    out = layers.Dense(1, name='y')(x)
    return keras.Model(inp, out, name='FFNN')


def make_cnn_fusion(n_q: int, max_visits: int, tab_dim: int) -> keras.Model:
    # matrix branch
    mat_in = layers.Input(shape=(n_q, max_visits, 1), name='alsfrs_mat')
    x = layers.Conv2D(16, kernel_size=(n_q, 3), activation='relu', padding='valid')(mat_in)  # (1, max_visits-2, 16)
    x = layers.Conv2D(32, kernel_size=(1, 3), activation='relu', padding='valid')(x)        # (1, max_visits-4, 32)
    x = layers.GlobalMaxPooling2D()(x)

    # tabular non-ALS branch
    tab_in = layers.Input(shape=(tab_dim,), name='non_als_tab')

    h = layers.Concatenate()([x, tab_in])
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dropout(0.3)(h)
    out = layers.Dense(1, name='y')(h)

    return keras.Model([mat_in, tab_in], out, name='CNN_Fusion')


def make_rnn_fusion(n_q: int, max_visits: int, tab_dim: int, cell: str = 'LSTM') -> keras.Model:
    mat_in = layers.Input(shape=(n_q, max_visits, 1), name='alsfrs_mat')
    # reshape to (time, features) = (max_visits, n_q)
    x = layers.Lambda(lambda t: tf.squeeze(t, axis=-1))(mat_in)  # (n_q, max_visits)
    x = layers.Permute((2, 1))(x)  # (max_visits, n_q)

    if cell.upper() == 'GRU':
        x = layers.GRU(64, return_sequences=True)(x)
        x = layers.GRU(32)(x)
    else:
        x = layers.LSTM(64, return_sequences=True)(x)
        x = layers.LSTM(32)(x)

    tab_in = layers.Input(shape=(tab_dim,), name='non_als_tab')
    h = layers.Concatenate()([x, tab_in])
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dropout(0.3)(h)
    out = layers.Dense(1, name='y')(h)

    return keras.Model([mat_in, tab_in], out, name='RNN_Fusion')


# =============================
# Main training & evaluation
# =============================

def run_pipeline(bootstrap_n: int = 10000, verbose: int = 1):
    # Sanity checks
    for var in ['X_train', 'X_test', 'y_train', 'y_test', 'features_df', 'alsfrs_3m']:
        if var not in globals():
            raise RuntimeError(f"Missing `{var}` in memory. Please run your Step 1 & Step 2 cells first.")

    set_global_seed(42)

    # ------------------
    # 1) FFNN baseline
    # ------------------
    if verbose:
        print("\n[1] Training FFNN baseline on selected summary features (tabular only)...")
    ffnn = make_ffnn(input_dim=X_train.shape[1])

    # validation split from training set
    hist = compile_and_fit(
        ffnn,
        X_train, y_train,
        X_val=X_test, y_val=y_test,  # simple holdout val; for hyperparam search, wrap with KFold CV
        epochs=300, batch_size=64, patience=20, verbose=verbose
    )

    y_pred_ffnn = ffnn.predict(X_test, verbose=0).reshape(-1)
    ffnn_rmsd, ffnn_ci = bootstrap_ci(y_test.values, y_pred_ffnn, rmsd, n_boot=bootstrap_n)
    ffnn_p, ffnn_p_ci = bootstrap_ci(y_test.values, y_pred_ffnn, pcc, n_boot=bootstrap_n)

    if verbose:
        print(f"FFNN  → RMSD: {ffnn_rmsd:.3f}  95% CI{ffnn_ci} | PCC: {ffnn_p:.3f} 95% CI{ffnn_p_ci}")

    # -----------------------------------------
    # 2) Build ALSFRS visit-matrices for fusion nets
    # -----------------------------------------
    if verbose:
        print("\n[2] Building ALSFRS visit-matrices (n_q × 5) from alsfrs_3m...")

    patient_ids_all = features_df.index.tolist()
    A_all, qcols = build_alsfrs_matrices(alsfrs_3m, patient_ids_all, max_visits=5, time_col='ALSFRS_Delta')

    # Align with X_train/X_test by index order
    id_to_pos = {pid: i for i, pid in enumerate(patient_ids_all)}
    idx_tr = [id_to_pos[i] for i in X_train.index]
    idx_te = [id_to_pos[i] for i in X_test.index]

    A_tr = A_all[idx_tr]
    A_te = A_all[idx_te]

    # Non‑ALS tabular branch: drop columns that appear to be ALSFRS-derived (prefix 'Q')
    non_als_cols = [c for c in X_train.columns if not c.startswith('Q')]
    Xtr_nonals = X_train[non_als_cols].copy()
    Xte_nonals = X_test[non_als_cols].copy()

    if verbose:
        print(f"   - Question cols (n_q): {len(qcols)}: {qcols}")
        print(f"   - Non‑ALS tabular dim: {Xtr_nonals.shape[1]}")

    # ---------------------
    # 3) CNN-fusion model
    # ---------------------
    if verbose:
        print("\n[3] Training CNN-fusion model (11×3 → 1×3 convs + FFNN head)...")
    cnn_f = make_cnn_fusion(n_q=A_tr.shape[1], max_visits=A_tr.shape[2], tab_dim=Xtr_nonals.shape[1])
    compile_and_fit(cnn_f, [A_tr, Xtr_nonals], y_train, [A_te, Xte_nonals], y_test, epochs=300, batch_size=64, patience=20, verbose=verbose)

    y_pred_cnn = cnn_f.predict([A_te, Xte_nonals], verbose=0).reshape(-1)
    cnn_rmsd, cnn_ci = bootstrap_ci(y_test.values, y_pred_cnn, rmsd, n_boot=bootstrap_n)
    cnn_p, cnn_p_ci = bootstrap_ci(y_test.values, y_pred_cnn, pcc, n_boot=bootstrap_n)
    if verbose:
        print(f"CNN-F → RMSD: {cnn_rmsd:.3f}  95% CI{cnn_ci} | PCC: {cnn_p:.3f} 95% CI{cnn_p_ci}")

    # ---------------------
    # 4) RNN-fusion model
    # ---------------------
    if verbose:
        print("\n[4] Training RNN-fusion model (2 recurrent layers + FFNN head)...")
    rnn_f = make_rnn_fusion(n_q=A_tr.shape[1], max_visits=A_tr.shape[2], tab_dim=Xtr_nonals.shape[1], cell='LSTM')
    compile_and_fit(rnn_f, [A_tr, Xtr_nonals], y_train, [A_te, Xte_nonals], y_test, epochs=300, batch_size=64, patience=20, verbose=verbose)

    y_pred_rnn = rnn_f.predict([A_te, Xte_nonals], verbose=0).reshape(-1)
    rnn_rmsd, rnn_ci = bootstrap_ci(y_test.values, y_pred_rnn, rmsd, n_boot=bootstrap_n)
    rnn_p, rnn_p_ci = bootstrap_ci(y_test.values, y_pred_rnn, pcc, n_boot=bootstrap_n)
    if verbose:
        print(f"RNN-F → RMSD: {rnn_rmsd:.3f}  95% CI{rnn_ci} | PCC: {rnn_p:.3f} 95% CI{rnn_p_ci}")

    # ---------------------
    # 5) Simple ensemble
    # ---------------------
    if verbose:
        print("\n[5] Ensemble (FFNN + CNN) — simple average of predictions...")
    ens_pred = 0.5 * y_pred_ffnn + 0.5 * y_pred_cnn
    ens_rmsd, ens_ci = bootstrap_ci(y_test.values, ens_pred, rmsd, n_boot=bootstrap_n)
    ens_p, ens_p_ci = bootstrap_ci(y_test.values, ens_pred, pcc, n_boot=bootstrap_n)
    if verbose:
        print(f"ENS   → RMSD: {ens_rmsd:.3f}  95% CI{ens_ci} | PCC: {ens_p:.3f} 95% CI{ens_p_ci}")

    return {
        'ffnn':  {'pred': y_pred_ffnn, 'rmsd': ffnn_rmsd, 'rmsd_ci': ffnn_ci, 'pcc': ffnn_p, 'pcc_ci': ffnn_p_ci},
        'cnn_f': {'pred': y_pred_cnn,  'rmsd': cnn_rmsd,  'rmsd_ci': cnn_ci,  'pcc': cnn_p,  'pcc_ci': cnn_p_ci},
        'rnn_f': {'pred': y_pred_rnn,  'rmsd': rnn_rmsd,  'rmsd_ci': rnn_ci,  'pcc': rnn_p,  'pcc_ci': rnn_p_ci},
        'ens':   {'pred': ens_pred,    'rmsd': ens_rmsd,  'rmsd_ci': ens_ci,  'pcc': ens_p,  'pcc_ci': ens_p_ci},
        'qcols': qcols,
        'non_als_cols': non_als_cols
    }


# Example usage (uncomment to run after your preprocessing cells):
# results = run_pipeline(bootstrap_n=10000, verbose=1)
# print(results)


# =============================
# PyTorch Training Module (Warmup + Cosine, RMSE-focus)
# Matches the user-provided training loop API but supports tuple inputs
# for fusion models (ALS matrix + non‑ALS tabular)
# =============================

import time, gc, math
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

# -------------------------------
# Config (example)
# -------------------------------
config = {
    'n_epochs': 300,
    'early_stopping_patience': 20,
    'batch_size': 64,
    'warmup_epochs': 10,
    'learning_rate_start': 1e-5,
    'learning_rate_max': 1e-3
}

_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure placeholder exists for artifact bundle
try:
    preprocessing_artifacts
except NameError:
    preprocessing_artifacts = {}

# -------------------------------
# Datasets
# -------------------------------
class TabularDS(Dataset):
    def __init__(self, X_df, y_s):
        self.X = torch.tensor(X_df.values, dtype=torch.float32)
        self.y = torch.tensor(y_s.values.reshape(-1, 1), dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class FusionDS(Dataset):
    def __init__(self, A_np, X_nonals_df, y_s):
        # A: (N, n_q, max_visits, 1) → to torch as (N, 1, n_q, max_visits)
        A = np.transpose(A_np, (0, 3, 1, 2))
        self.A = torch.tensor(A, dtype=torch.float32)
        self.T = torch.tensor(X_nonals_df.values, dtype=torch.float32)
        self.y = torch.tensor(y_s.values.reshape(-1, 1), dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        # Return ((als_mat, tab_nonals), y)
        return (self.A[idx], self.T[idx]), self.y[idx]

# -------------------------------
# Models (paper-faithful)
# -------------------------------
class FFNN_PT(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x)

class CNNFusion_PT(nn.Module):
    def __init__(self, n_q, max_visits, tab_dim):
        super().__init__()
        # Input A: (B, 1, n_q, max_visits)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(n_q, 3), padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(1, 3), padding=0)
        self.head = nn.Sequential(
            nn.Linear(32 + tab_dim, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        # x may be ((A, T)) or (A, T) or just A
        if isinstance(x, (list, tuple)):
            A, T = x
        else:
            A, T = x, None
        h = torch.relu(self.conv1(A))
        h = torch.relu(self.conv2(h))
        # Global max over H and W
        h = torch.amax(h, dim=(2, 3))  # (B, 32)
        if T is None:
            raise RuntimeError("CNNFusion expects (ALS_matrix, tabular_nonALS)")
        z = torch.cat([h, T], dim=1)
        return self.head(z)

class RNNFusion_PT(nn.Module):
    def __init__(self, n_q, max_visits, tab_dim, cell='LSTM'):
        super().__init__()
        self.cell = cell.upper()
        feat = n_q
        hid1, hid2 = 64, 32
        if self.cell == 'GRU':
            self.r1 = nn.GRU(input_size=feat, hidden_size=hid1, batch_first=True)
            self.r2 = nn.GRU(input_size=hid1, hidden_size=hid2, batch_first=True)
        else:
            self.r1 = nn.LSTM(input_size=feat, hidden_size=hid1, batch_first=True)
            self.r2 = nn.LSTM(input_size=hid1, hidden_size=hid2, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(hid2 + tab_dim, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        if isinstance(x, (list, tuple)):
            A, T = x
        else:
            A, T = x, None
        # A: (B, 1, n_q, max_visits) → (B, max_visits, n_q)
        A = torch.permute(A.squeeze(1), (0, 2, 1))
        o1, _ = self.r1(A)
        o2, _ = self.r2(o1)
        h = o2[:, -1, :]
        if T is None:
            raise RuntimeError("RNNFusion expects (ALS_matrix, tabular_nonALS)")
        z = torch.cat([h, T], dim=1)
        return self.head(z)

# -------------------------------
# LR Scheduler (Warmup + Cosine)
# -------------------------------

def get_lr(epoch, cfg):
    if epoch < cfg['warmup_epochs']:
        return cfg['learning_rate_start'] + (cfg['learning_rate_max'] - cfg['learning_rate_start']) * (epoch / cfg['warmup_epochs'])
    progress = (epoch - cfg['warmup_epochs']) / max(1, (cfg['n_epochs'] - cfg['warmup_epochs']))
    return cfg['learning_rate_max'] * 0.5 * (1 + math.cos(math.pi * progress))

# -------------------------------
# Train/Eval that support tuple inputs and device
# -------------------------------

def _to_device(batch_x, batch_y):
    if isinstance(batch_x, (list, tuple)):
        batch_x = tuple(t.to(_device) for t in batch_x)
    else:
        batch_x = batch_x.to(_device)
    batch_y = batch_y.to(_device)
    return batch_x, batch_y


def train_epoch(model, loader, criterion, optimizer, epoch, cfg):
    model.train()
    total_loss = 0.0
    lr = get_lr(epoch, cfg)
    for pg in optimizer.param_groups:
        pg['lr'] = lr

    for batch_x, batch_y in loader:
        batch_x, batch_y = _to_device(batch_x, batch_y)
        optimizer.zero_grad(set_to_none=True)
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
        optimizer.step()
        total_loss += loss.item()
        del pred, loss
    gc.collect()
    return total_loss / len(loader), lr


def evaluate(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x, batch_y = _to_device(batch_x, batch_y)
            pred = model(batch_x)
            preds.extend(pred.detach().cpu().numpy().reshape(-1))
            trues.extend(batch_y.detach().cpu().numpy().reshape(-1))
            del pred
    preds = np.array(preds)
    trues = np.array(trues)
    rmse = float(np.sqrt(mean_squared_error(trues, preds)))
    mae = float(mean_absolute_error(trues, preds))
    r2 = float(r2_score(trues, preds))
    pcc = float(pearsonr(trues, preds)[0]) if np.std(preds) > 0 and np.std(trues) > 0 else float('nan')
    return rmse, mae, r2, pcc, preds, trues

# -------------------------------
# Runner (as per provided printouts)
# -------------------------------

def run_training(model, train_loader, test_loader, cfg):
    print("=" * 60)
    print("STEP 4: BREAKTHROUGH TRAINING")
    print("=" * 60)
    print("\n🚀 Starting breakthrough training...")
    print(f"   Target: RMSE < 0.35, PCC > 0.65")
    print(f"   Strategy: Warmup + Cosine decay + Heavy RMSE focus")
    print(f"   Epochs: {cfg['n_epochs']}")
    print(f"   Patience: {cfg['early_stopping_patience']}")
    print("-" * 60)

    model = model.to(_device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg['learning_rate_start'])

    best_score = -float('inf')
    best_rmse = float('inf')
    best_pcc = -1
    best_mae = float('inf')
    best_r2 = -1
    patience_counter = 0
    start_time = time.time()

    history = {
        'train_loss': [],
        'test_rmse': [],
        'test_pcc': [],
        'learning_rates': []
    }

    for epoch in range(cfg['n_epochs']):
        epoch_start = time.time()
        train_loss, lr = train_epoch(model, train_loader, criterion, optimizer, epoch, cfg)
        rmse, mae, r2, pcc, _, _ = evaluate(model, test_loader)

        history['train_loss'].append(train_loss)
        history['test_rmse'].append(rmse)
        history['test_pcc'].append(pcc)
        history['learning_rates'].append(lr)

        combined_score = -2.0 * rmse + pcc
        if combined_score > best_score:
            best_score = combined_score
            best_rmse = rmse
            best_pcc = pcc
            best_mae = mae
            best_r2 = r2
            best_model_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1

        epoch_time = time.time() - epoch_start
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:3d}/{cfg['n_epochs']} | Loss: {train_loss:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | PCC: {pcc:.4f} | R²: {r2:.4f} | LR: {lr:.6f} | Time: {epoch_time:.1f}s")

        if patience_counter >= cfg['early_stopping_patience']:
            print(f"\n⏹️  Early stopping triggered at epoch {epoch+1}")
            print(f"   No improvement for {cfg['early_stopping_patience']} epochs")
            break

    total_time = time.time() - start_time
    print("-" * 60)
    print(f"✅ Training completed in {total_time:.1f}s ({total_time/60:.1f} min)")

    print("\n📊 Loading best model and final evaluation...")
    model.load_state_dict(best_model_state)
    model = model.to(_device)
    final_rmse, final_mae, final_r2, final_pcc, predictions, actuals = evaluate(model, test_loader)

    print("\n" + "=" * 60)
    print("🏆 BREAKTHROUGH RESULTS")
    print("=" * 60)
    print(f"\n📊 Final Test Performance:")
    print(f"   RMSE: {final_rmse:.4f}")
    print(f"   MAE: {final_mae:.4f}")
    print(f"   R² Score: {final_r2:.4f}")
    print(f"   Pearson Correlation (PCC): {final_pcc:.4f}")

    print(f"\n📈 Performance vs Targets:")
    print(f"   Target RMSE: < 0.35")
    print(f"   Achieved RMSE: {final_rmse:.4f}")
    if final_rmse < 0.35:
        improvement = ((0.58 - final_rmse) / 0.58) * 100
        print(f"   ✅ RMSE TARGET ACHIEVED!")
        print(f"   🎉 {improvement:.1f}% improvement from original (0.58)")
    else:
        gap = final_rmse - 0.35
        reduction_from_041 = ((0.41 - final_rmse) / 0.41) * 100
        print(f"   📍 Gap to target: {gap:.4f}")
        print(f"   📉 Reduced from 0.41: {reduction_from_041:.1f}%")

    print(f"\n   Target PCC: > 0.65")
    print(f"   Achieved PCC: {final_pcc:.4f}")
    if final_pcc > 0.65:
        print(f"   ✅ PCC TARGET ACHIEVED!")
    else:
        gap = 0.65 - final_pcc
        print(f"   📍 Gap to target: {gap:.4f}")

    print(f"\n📊 Training Statistics:")
    print(f"   Total epochs: {len(history['train_loss'])}")
    print(f"   Best RMSE: {best_rmse:.4f}")
    print(f"   Best PCC: {best_pcc:.4f}")
    print(f"   Best MAE: {best_mae:.4f}")
    print(f"   Best R²: {best_r2:.4f}")
    print(f"   Final LR: {history['learning_rates'][-1]:.6f}")
    print(f"   Training time: {total_time/60:.1f} minutes")

    errors = actuals - predictions
    print("\n🔍 Detailed Prediction Analysis (first 15 samples):")
    print(f"{'Actual':>10} | {'Predicted':>10} | {'Error':>10} | {'% Error':>10}")
    print("-" * 50)
    for i in range(min(15, len(actuals))):
        a = actuals[i]; p = predictions[i]
        err = a - p; pct = (err / a * 100) if a != 0 else 0
        print(f"{a:>10.4f} | {p:>10.4f} | {err:>10.4f} | {pct:>9.1f}%")

    print("\n📊 Error Statistics:")
    print(f"   Mean error: {errors.mean():.4f}")
    print(f"   Std error: {errors.std():.4f}")
    print(f"   Max overestimation: {errors.max():.4f}")
    print(f"   Max underestimation: {errors.min():.4f}")
    print(f"   Median absolute error: {np.median(np.abs(errors)):.4f}")

    final_artifacts = {
        'model_state': best_model_state,
        'config': cfg,
        'preprocessing': preprocessing_artifacts,
        'history': history,
        'best_metrics': {
            'rmse': best_rmse,
            'mae': best_mae,
            'r2': best_r2,
            'pcc': best_pcc
        }
    }

    print("\n" + "=" * 60)
    print("✅ STEP 4 COMPLETED SUCCESSFULLY")
    print("=" * 60)
    print("\n🎉 BREAKTHROUGH TRAINING COMPLETE!")
    print(f"\n🎯 Final Summary:")
    print(f"   This Model:      RMSE={final_rmse:.4f}, PCC={final_pcc:.4f}")
    return final_artifacts, (predictions, actuals)

# -------------------------------
# Wiring for each paper model
# -------------------------------
# Pre-req from earlier cells: X_train, X_test, y_train, y_test, features_df, alsfrs_3m

# Build ALSFRS matrices aligned with splits (reuse earlier function build_alsfrs_matrices)
patient_ids_all = features_df.index.tolist()
A_all, qcols = build_alsfrs_matrices(alsfrs_3m, patient_ids_all, max_visits=5, time_col='ALSFRS_Delta')
idx_map = {pid: i for i, pid in enumerate(patient_ids_all)}
idx_tr = [idx_map[i] for i in X_train.index]
idx_te = [idx_map[i] for i in X_test.index]
A_tr = A_all[idx_tr]
A_te = A_all[idx_te]

non_als_cols = [c for c in X_train.columns if not c.startswith('Q')]
Xtr_nonals = X_train[non_als_cols].copy()
Xte_nonals = X_test[non_als_cols].copy()

# DataLoaders
train_loader_ff = DataLoader(TabularDS(X_train, y_train), batch_size=config['batch_size'], shuffle=True, drop_last=False)
test_loader_ff  = DataLoader(TabularDS(X_test, y_test),   batch_size=config['batch_size'], shuffle=False, drop_last=False)

train_loader_cnn = DataLoader(FusionDS(A_tr, Xtr_nonals, y_train), batch_size=config['batch_size'], shuffle=True, drop_last=False)
test_loader_cnn  = DataLoader(FusionDS(A_te, Xte_nonals, y_test),   batch_size=config['batch_size'], shuffle=False, drop_last=False)

train_loader_rnn = train_loader_cnn  # same inputs
test_loader_rnn  = test_loader_cnn

# Instantiate models
ffnn_pt = FFNN_PT(in_dim=X_train.shape[1])
cnn_f_pt = CNNFusion_PT(n_q=A_tr.shape[1], max_visits=A_tr.shape[2], tab_dim=Xtr_nonals.shape[1])
rnn_f_pt = RNNFusion_PT(n_q=A_tr.shape[1], max_visits=A_tr.shape[2], tab_dim=Xtr_nonals.shape[1], cell='LSTM')

# === RUN ===
ffnn_artifacts, _ = run_training(ffnn_pt, train_loader_ff, test_loader_ff, config)
# cnn_artifacts,  _ = run_training(cnn_f_pt, train_loader_cnn, test_loader_cnn, config)
# rnn_artifacts,  _ = run_training(rnn_f_pt, train_loader_rnn, test_loader_rnn, config)


STEP 4: BREAKTHROUGH TRAINING

🚀 Starting breakthrough training...
   Target: RMSE < 0.35, PCC > 0.65
   Strategy: Warmup + Cosine decay + Heavy RMSE focus
   Epochs: 300
   Patience: 20
------------------------------------------------------------
Epoch   1/300 | Loss: 0.4430 | RMSE: 0.6017 | MAE: 0.3981 | PCC: -0.0788 | R²: -0.5374 | LR: 0.000010 | Time: 0.6s
Epoch   5/300 | Loss: 0.2103 | RMSE: 0.4221 | MAE: 0.2948 | PCC: 0.5223 | R²: 0.2435 | LR: 0.000406 | Time: 0.6s
Epoch  10/300 | Loss: 0.1692 | RMSE: 0.4018 | MAE: 0.2772 | PCC: 0.5661 | R²: 0.3146 | LR: 0.000901 | Time: 0.4s
Epoch  15/300 | Loss: 0.1640 | RMSE: 0.4056 | MAE: 0.2768 | PCC: 0.5593 | R²: 0.3013 | LR: 0.001000 | Time: 0.4s
Epoch  20/300 | Loss: 0.1537 | RMSE: 0.4058 | MAE: 0.2825 | PCC: 0.5497 | R²: 0.3007 | LR: 0.000998 | Time: 0.4s
Epoch  25/300 | Loss: 0.1534 | RMSE: 0.4063 | MAE: 0.2772 | PCC: 0.5563 | R²: 0.2990 | LR: 0.000994 | Time: 0.4s
Epoch  30/300 | Loss: 0.1487 | RMSE: 0.4086 | MAE: 0.2824 | PCC: 0.5429 